In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc
gc.collect()

In [ ]:
#import singleton
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS
import read_data as imdbImport

LESSON_DATA_FOLDER = imdbImport.LESSON_DATA_FOLDER

In [ ]:
#putil = ParallelUtil('./data/parallel.config.tsv')

In [ ]:
crew = imdbImport.readFile('merged.names.principals.tsv', nrows = None)
crew.info()

In [ ]:
crew.tail()

In [ ]:
crew_grouped = imdbImport.readFile('merged.names.principals.tsv', nrows = None).groupby(['tconst'])
#crew.groupby(['tconst'])



'''[['tconst','nconst','ordering','category',
                                        'job','characters','primaryName','birthYear','deathYear',
                                        'wiki','isAlive'
                                       ]]
'''                                       

In [ ]:
crew_grouped.get_group('tt0000001')

In [ ]:
titles = imdbImport.readFile('merged.title.ratings.tsv', nrows = None)
titles.info()

In [ ]:
#@dview.parallel(block=True)
def f(group):    
    group = group.set_index('tconst')        
    return pd.DataFrame(data={'crew': group.to_json(orient='records')}, index = pd.Series(group.index.values).unique()  )  
    
crew_grouped = crew.groupby(['tconst']).apply(f)
print ('Crew groupby apply done: {}'.format(time.strftime("%Y-%m-%d %H:%M:%S"))  )

crew_grouped = crew_grouped.reset_index().drop(['level_1'], axis= 1)
print ('Crew axis drop done: {}'.format(time.strftime("%Y-%m-%d %H:%M:%S"))  )

merged_title_crew = pd.merge(titles, crew_grouped, on = ['tconst'], how = 'left')
print ('Merge done: {}'.format(time.strftime("%Y-%m-%d %H:%M:%S"))  )

In [ ]:
merged_title_crew.info()
#crew_grouped_1 = crew_grouped_1.reset_index()

In [ ]:
merged_title_crew

In [ ]:
#titles.head(2).merge(crew_grouped_1, left_on =['tconst'], how='left', left_index= True)
#merged_title_crew = pd.merge(titles.head(2), crew_grouped_1, on = ['tconst'])

In [ ]:
merged_title_crew.to_json(''.join([LESSON_DATA_FOLDER, 'merged.title.crew.json']), orient='records')

In [1]:
from helper.imdb_sqllite_db import IMDBSQLLite

In [2]:
sqlConn = IMDBSQLLite()

IMDB SQLLite Database: ./data/imdb.sqllite


In [ ]:
sqlConn.cursor